In [84]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# RESOLVER PROBLEMA DE VERSÃO PYTHON X PYSPARK WORKERS

In [85]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *

from pyspark.sql.types import *

import pandas as pd

In [86]:
spark = SparkSession.builder \
    .master("local") \
    .appName("pyspark_01") \
    .getOrCreate()    

In [87]:
df = spark.read.csv("players.csv", header=True, inferSchema=True)
df.printSchema()
df.show()

root
 |-- Team: string (nullable = true)
 |-- #: integer (nullable = true)
 |-- Pos.: string (nullable = true)
 |-- FIFA Popular Name: string (nullable = true)
 |-- Birth Date: string (nullable = true)
 |-- Shirt Name: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- Height: integer (nullable = true)
 |-- Weight: integer (nullable = true)

+---------+---+----+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
|Argentina| 18|  DF|    SALVIO Eduardo|13.07.1990|    SALVI

In [88]:
# não é aconselhavel usar. pode travar
df.toPandas().isnull().sum()

Team                 0
#                    0
Pos.                 0
FIFA Popular Name    0
Birth Date           0
Shirt Name           0
Club                 0
Height               0
Weight               0
dtype: int64

In [92]:
for coluna in df.columns:
    print(coluna, df.filter(df[coluna].isNull()).count())

Selecao 0
Número 0
Posicao 0
Nome_FiFA 0
Nascimento 0
Nome Camiseta 0
Time 0
Altura 0
Peso 0


In [90]:
df.show()

+---------+---+----+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
|Argentina| 18|  DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69|
|Argentina| 10|  FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72|
|Argentina|  4|  DF|  ANSALDI Cristian|20.09.1986|   ANSALDI|     Torino FC (ITA)|   181|    73|
|Argentina|  5|  MF|      BIGLIA Lucas|30.01.1986|    BIGLIA|      AC Milan (ITA)|   175|    73|
|Argentina|  7|  MF|       BAN

<h3>renomear colunas</h3>

In [91]:
# renomear colunas só pode fazer isso no pandas no spark é imutavel
df = df.withColumnRenamed("Team", "Selecao").withColumnRenamed("#", "Número")\
.withColumnRenamed("Pos.", "Posicao").withColumnRenamed("FIFA Popular Name", "Nome_FiFA")\
.withColumnRenamed("Birth Date", "Nascimento").withColumnRenamed("Shirt Name", "Nome Camiseta")\
.withColumnRenamed("Club", "Time").withColumnRenamed("Height", "Altura").withColumnRenamed("Weight", "Peso")

df.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|  Selecao|Número|Posicao|         Nome_FiFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
only showing top 5 rows



In [95]:
df.columns

['Selecao',
 'Número',
 'Posicao',
 'Nome_FiFA',
 'Nascimento',
 'Nome Camiseta',
 'Time',
 'Altura',
 'Peso']

In [93]:
for coluna in df.columns:
    print(coluna, df.filter(df[coluna].isNull()).count())

Selecao 0
Número 0
Posicao 0
Nome_FiFA 0
Nascimento 0
Nome Camiseta 0
Time 0
Altura 0
Peso 0


In [98]:
# Selecionar colunas
df.select('Selecao', 'nome_fifa').show(5)

+---------+------------------+
|  Selecao|         nome_fifa|
+---------+------------------+
|Argentina|TAGLIAFICO Nicolas|
|Argentina|    PAVON Cristian|
|Argentina|    LANZINI Manuel|
|Argentina|    SALVIO Eduardo|
|Argentina|      MESSI Lionel|
+---------+------------------+
only showing top 5 rows



In [99]:
df.select(col('selecao')).show(5)

+---------+
|  selecao|
+---------+
|Argentina|
|Argentina|
|Argentina|
|Argentina|
|Argentina|
+---------+
only showing top 5 rows



In [101]:
df.select(df['selecao']).show(5)

+---------+
|  selecao|
+---------+
|Argentina|
|Argentina|
|Argentina|
|Argentina|
|Argentina|
+---------+
only showing top 5 rows



In [104]:
# Criar Alias
df.select(col('selecao').alias('Time')).show(5)

+---------+
|     Time|
+---------+
|Argentina|
|Argentina|
|Argentina|
|Argentina|
|Argentina|
+---------+
only showing top 5 rows



In [105]:
#organizar select
df.select('nome_fifa', 'Peso', 'Altura').show(5)

+------------------+----+------+
|         nome_fifa|Peso|Altura|
+------------------+----+------+
|TAGLIAFICO Nicolas|  65|   169|
|    PAVON Cristian|  65|   169|
|    LANZINI Manuel|  66|   167|
|    SALVIO Eduardo|  69|   167|
|      MESSI Lionel|  72|   170|
+------------------+----+------+
only showing top 5 rows



In [107]:
#Filtrar DataFrame
df.filter('selecao = "Brazil"').show(5)

+-------+------+-------+-----------+----------+-------------+--------------------+------+----+
|Selecao|Número|Posicao|  Nome_FiFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+-------+------+-------+-----------+----------+-------------+--------------------+------+----+
| Brazil|    18|     MF|       FRED|05.03.1993|         FRED|FC Shakhtar Donet...|   169|  64|
| Brazil|    21|     FW|     TAISON|13.01.1988|       TAISON|FC Shakhtar Donet...|   172|  64|
| Brazil|    17|     MF|FERNANDINHO|04.05.1985|  FERNANDINHO|Manchester City F...|   179|  67|
| Brazil|    22|     DF|     FAGNER|11.06.1989|       FAGNER|SC Corinthians (BRA)|   168|  67|
| Brazil|    10|     FW|     NEYMAR|05.02.1992|    NEYMAR JR|Paris Saint-Germa...|   175|  68|
+-------+------+-------+-----------+----------+-------------+--------------------+------+----+
only showing top 5 rows



In [111]:
# outra formar de filtrar caso tenha espaço no nome da coluna
df.filter(col('Nome camiseta') == "FRED").show()

+-------+------+-------+---------+----------+-------------+--------------------+------+----+
|Selecao|Número|Posicao|Nome_FiFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+-------+------+-------+---------+----------+-------------+--------------------+------+----+
| Brazil|    18|     MF|     FRED|05.03.1993|         FRED|FC Shakhtar Donet...|   169|  64|
+-------+------+-------+---------+----------+-------------+--------------------+------+----+



In [112]:
#Filtrar 2 condições (and / &)
df.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|  Selecao|Número|Posicao|         Nome_FiFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
only showing top 5 rows



In [123]:
df.filter((col('selecao') == "Argentina") & (col('Altura') < 165)).show()

+-------+------+-------+---------+----------+-------------+----+------+----+
|Selecao|Número|Posicao|Nome_FiFA|Nascimento|Nome Camiseta|Time|Altura|Peso|
+-------+------+-------+---------+----------+-------------+----+------+----+
+-------+------+-------+---------+----------+-------------+----+------+----+



In [125]:
df.show()
df.filter((col('selecao') == "Brazil"))

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|  Selecao|Número|Posicao|         Nome_FiFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|
|Argentina|     4|     DF|  ANSALDI Cristian|20.09.1986|      ANSALDI|     Torino FC (ITA)|   181|  73|
|Argentina|     5|     MF|      BIGLIA Lucas|30.01.1986|       B